## Requirement

In [ ]:
!pip install lazypredict

In [4]:
import json
import pandas as pd

from lazypredict.Supervised import LazyClassifier,LazyRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Data loading

In [5]:
file_match_1 = open('/content/match_1.json')
data = json.load(file_match_1)
df_match=pd.DataFrame(data)
df_match

,label,norm
0,walk,"[23.177037336396975, 23.36152528524938, 21.535..."
1,walk,"[20.998214383911275, 19.182798059840767, 18.27..."
2,walk,"[21.95259682019565, 20.62720484424047, 22.5554..."
3,walk,"[19.39209748358647, 19.30460665293087, 18.9787..."
4,rest,"[22.069263037713093, 19.672270483203395, 19.88..."
...,...,...
569,walk,"[21.245737438782683, 21.938836193196945, 21.24..."
570,walk,"[20.272347748199333, 21.820302584302983, 21.21..."
571,run,"[90.21306005395827, 72.12712844635696, 63.5643..."
572,tackle,"[44.79510251441832, 27.26406867633046, 88.6164..."


In [6]:
df_match['sequence_length'] = df_match['norm'].apply(len)

In [7]:
 df_match =df_match.drop(df_match[(df_match['sequence_length'] < 10) | (df_match['sequence_length'] >300) ].index)

In [8]:
df_match

,label,norm,sequence_length
0,walk,"[23.177037336396975, 23.36152528524938, 21.535...",72
1,walk,"[20.998214383911275, 19.182798059840767, 18.27...",54
2,walk,"[21.95259682019565, 20.62720484424047, 22.5554...",68
3,walk,"[19.39209748358647, 19.30460665293087, 18.9787...",75
4,rest,"[22.069263037713093, 19.672270483203395, 19.88...",181
...,...,...,...
569,walk,"[21.245737438782683, 21.938836193196945, 21.24...",85
570,walk,"[20.272347748199333, 21.820302584302983, 21.21...",48
571,run,"[90.21306005395827, 72.12712844635696, 63.5643...",32
572,tackle,"[44.79510251441832, 27.26406867633046, 88.6164...",64


In [9]:
label_encoder= LabelEncoder()
label_encoder.fit(df_match['label'])
df_match['label'] = label_encoder.transform(df_match['label'])

In [10]:
df_match

,label,norm,sequence_length
0,7,"[23.177037336396975, 23.36152528524938, 21.535...",72
1,7,"[20.998214383911275, 19.182798059840767, 18.27...",54
2,7,"[21.95259682019565, 20.62720484424047, 22.5554...",68
3,7,"[19.39209748358647, 19.30460665293087, 18.9787...",75
4,3,"[22.069263037713093, 19.672270483203395, 19.88...",181
...,...,...,...
569,7,"[21.245737438782683, 21.938836193196945, 21.24...",85
570,7,"[20.272347748199333, 21.820302584302983, 21.21...",48
571,4,"[90.21306005395827, 72.12712844635696, 63.5643...",32
572,6,"[44.79510251441832, 27.26406867633046, 88.6164...",64


In [11]:
df_match['precedent_label'] = df_match['label'].shift()
df_match = df_match.iloc[1:]

In [12]:
df_match

,label,norm,sequence_length,precedent_label
1,7,"[20.998214383911275, 19.182798059840767, 18.27...",54,7.00
2,7,"[21.95259682019565, 20.62720484424047, 22.5554...",68,7.00
3,7,"[19.39209748358647, 19.30460665293087, 18.9787...",75,7.00
4,3,"[22.069263037713093, 19.672270483203395, 19.88...",181,7.00
5,7,"[19.92032356700904, 19.672270483203395, 18.688...",56,3.00
...,...,...,...,...
569,7,"[21.245737438782683, 21.938836193196945, 21.24...",85,7.00
570,7,"[20.272347748199333, 21.820302584302983, 21.21...",48,7.00
571,4,"[90.21306005395827, 72.12712844635696, 63.5643...",32,7.00
572,6,"[44.79510251441832, 27.26406867633046, 88.6164...",64,4.00


In [13]:

df_match['precedent_label'] = df_match['precedent_label'].astype(int)

In [14]:
df_match

,label,norm,sequence_length,precedent_label
1,7,"[20.998214383911275, 19.182798059840767, 18.27...",54,7
2,7,"[21.95259682019565, 20.62720484424047, 22.5554...",68,7
3,7,"[19.39209748358647, 19.30460665293087, 18.9787...",75,7
4,3,"[22.069263037713093, 19.672270483203395, 19.88...",181,7
5,7,"[19.92032356700904, 19.672270483203395, 18.688...",56,3
...,...,...,...,...
569,7,"[21.245737438782683, 21.938836193196945, 21.24...",85,7
570,7,"[20.272347748199333, 21.820302584302983, 21.21...",48,7
571,4,"[90.21306005395827, 72.12712844635696, 63.5643...",32,7
572,6,"[44.79510251441832, 27.26406867633046, 88.6164...",64,4


In [15]:
df_match['timestep_in_list']=[list(range(element)) for element in df_match['sequence_length']]

In [16]:
df_match

,label,norm,sequence_length,precedent_label,timestep_in_list
1,7,"[20.998214383911275, 19.182798059840767, 18.27...",54,7,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
2,7,"[21.95259682019565, 20.62720484424047, 22.5554...",68,7,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
3,7,"[19.39209748358647, 19.30460665293087, 18.9787...",75,7,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
4,3,"[22.069263037713093, 19.672270483203395, 19.88...",181,7,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
5,7,"[19.92032356700904, 19.672270483203395, 18.688...",56,3,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
...,...,...,...,...,...
569,7,"[21.245737438782683, 21.938836193196945, 21.24...",85,7,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
570,7,"[20.272347748199333, 21.820302584302983, 21.21...",48,7,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
571,4,"[90.21306005395827, 72.12712844635696, 63.5643...",32,7,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
572,6,"[44.79510251441832, 27.26406867633046, 88.6164...",64,4,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."


In [17]:
df_transformed = df_match.explode(['norm','timestep_in_list'])

In [18]:
df_transformed

,label,norm,sequence_length,precedent_label,timestep_in_list
1,7,21.00,54,7,0
1,7,19.18,54,7,1
1,7,18.27,54,7,2
1,7,19.69,54,7,3
1,7,20.83,54,7,4
...,...,...,...,...,...
573,4,19.88,36,6,31
573,4,11.20,36,6,32
573,4,24.70,36,6,33
573,4,30.05,36,6,34


In [19]:
y=df_transformed['norm']
x=df_transformed.drop(columns='norm')
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.5,random_state=42)

In [20]:
X_train

,label,sequence_length,precedent_label,timestep_in_list
392,4,28,4,26
476,4,62,2,52
181,7,54,7,4
471,7,54,7,3
538,3,82,7,27
...,...,...,...,...
457,7,62,7,23
133,7,44,7,1
18,4,38,4,2
354,4,64,2,56


In [21]:


regressor=RandomForestRegressor(n_estimators=100)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)
accuracy = regressor.score(X_test, y_test)

In [22]:
accuracy

0.18706708412712814